In [1]:
import pandas as pd
import json
from ParameterValues_MonteCarlo import ParameterValues
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
options = ['buttonGHG', 'buttonWithWater', 'buttonConsWater']
model = 'buttonConsWater'

In [13]:
SuperPro_names = {
    "Corn Liquor": "csl.kg",
    "Diammonium phos": "dap.kg",
    "EMIM Acetate": "ionicLiquid_amount",
    "ChLy": "ionicLiquid_amount",
    "Hydrolase": "cellulase_amount",
    "Methane": "ng_input_stream_MJ",
    "NaOH (50% w/w)": "naoh.kg",
    "Octane": "octane_ltr",
    "Std Power": "electricity",
    "Stover": "feedstock.kg",
    "WWT nutrients": "caoh.kg",
    "Sulfuric Acid": "acid.kg",
    "Water": "water_direct_consumption",
    "Hydrogen": "h2.kg",
    "Inoculum": "inoculum.kg",
    "CIP2": "cip2.kg",
    "AlCl3": "alcl3.kg", 
    "dehydrating Cat": "cat.kg",
    "Ru": "ru.kg",
    "Pd/AC Catalyst": "PdAC_catalyst.kg",
    "Cooling Water": "cooling_water",
    "CoolingWater25C": "cooling_water25",
    "Chilled Water": "chilled_water",
    "Steam 180C": "steam_low",
    "Steam 226C": "steam_low",
    "Steam330C": "steam_high",
    "electricity_generated": "electricity_generated"
}

sections_translate = {  "Feedstock supply logistics": "Feedstock_Supply_Logistics",
                        "Feedstock handling": "Feedstock_Handling_and_Preparation", 
                        "Pretreatment": "IL_Pretreatment", 
                        "Hydrolysis and fermentation": "Enzymatic_Hydrolysis_and_Fermentation", 
                        "Wastewater treatment": "Wastewater_Treatment",
                        "Hydrogenation": "Hydrogeneration_and_Oligomerization",
                        "Recovery and separation": "Recovery_and_Separation", 
                        "Lignin utilization": "Lignin_Utilization",
                     "Credits": "Credits"}


fuels = ['1,8-cineole', 'Bisabolene', 'Epi-isozizaene', 'Limonene', 'Linalool']
fuels = ['Isopentenol']

In [14]:
def convertDfToDict(data):
    final_dict = {}
    grouped = data.groupby(data.columns[0])
    for process in processes:
        new_process_name = sections_translate[process]
        d={}
        for i, row in grouped.get_group(process).iterrows():
                            
            if row['Material'] not in SuperPro_names.keys():
                new_material_name = row['Material']
            else:
                new_material_name = SuperPro_names[row['Material']]
            
            if new_material_name in d.keys():
                row['Baseline']+=d[new_material_name]['Baseline']
                
            
            if 'Methane' in row['Material']:
                d.update({new_material_name:{'Baseline': row['Baseline'],
                                              'Minimum': row['Minimum '],
                                             'Maximum': row['Maximum'],
                                             'Std_Dev': row['Std_Dev']}})
            
            else:
                d.update({new_material_name:{'Baseline': row['Baseline'],
                                              'Minimum': row['Minimum '],
                                             'Maximum': row['Maximum'],
                                             'Std_Dev': row['Std_Dev']}})
        final_dict.update({new_process_name:d})
    return final_dict

In [15]:
for fuel in fuels:
    data = pd.read_csv('data_isopentenol/Risk/{}.csv'.format(fuel))
    processes = list(data['Process'].unique())
    risk_params = convertDfToDict(data)
    path = 'static_isopentenol/scenarios/SuperPro_data_Aerobic_Cholinium lysinate_{}_BY_ICR.js'.format(fuel)
    parameters = json.load(open(path))
    
    Params = ParameterValues(parameters, risk_params)
    sensi_params = Params.sensitivity(fuel, 'ChLys', model)
    
    final_df = pd.DataFrame()
    for process in sensi_params.keys():
        datafr = pd.DataFrame.from_dict(sensi_params[process]).T
        datafr['Process'] = process
        final_df = final_df.append(datafr)
    final_df=final_df.reset_index()
    final_df = final_df.rename(index=str, columns={'index': 'Material'})
    final_df.to_csv('results_isopentenol/sensitivity_{}_{}.csv'.format(fuel, model))

In [16]:
for fuel in fuels:
    final_df = pd.DataFrame()
    data = pd.read_csv('data_isopentenol/Risk/{}.csv'.format(fuel))
    processes = list(data['Process'].unique())
    risk_params = convertDfToDict(data)
    path = 'static_isopentenol/scenarios/SuperPro_data_Aerobic_Cholinium lysinate_{}_BY_ICR.js'.format(fuel)
    parameters = json.load(open(path))
    Params = ParameterValues(parameters, risk_params)
    for i in range(5000):
        uncertain_df = Params.uncertainty(fuel, 'ChLys', model)
        final_df = final_df.append(uncertain_df)
    final_df.to_csv('results_isopentenol/uncertainty_{}_{}.csv'.format(fuel, model))

In [ ]:
final_df['Total_gCO2_MJ_net'].hist(bins=100)